In [ ]:
import os
import json
import faiss
from Libraries import A0_Widgets
from Libraries import A1_Define
from Libraries import B1_ExtractData, B2_MergeData
from Libraries import B3_ChunkMaster, B4_ChunkFixed, B5_ChunkFlex

In [ ]:
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
widgets_list = A0_Widgets.create_name_form()

In [ ]:
config = A1_Define.WidgetValues(widgets_list)

data_foler = config["data_folder"]
dcmt_path = config["dcmt_path"]
base_folder = config["base_folder"]
base_path = config["base_path"]
chunks_base = config["chunks_base"]
json_file_path = config["json_file_path"]
schema_ex_path = config["schema_ex_path"]
embedding_path = config["embedding_path"]
torch_path = config["torch_path"]
faiss_path = config["faiss_path"]
mapping_path = config["mapping_path"]
mapping_data = config["mapping_data"]

FILE_TYPE = config["FILE_TYPE"]
DATA_KEY = config["DATA_KEY"]
EMBE_KEY = config["EMBE_KEY"]
SWITCH = config["SWITCH"]
EMBEDD_MODEL = config["EMBEDD_MODEL"]
SEARCH_EGINE = config["SEARCH_EGINE"]
RERANK_MODEL = config["RERANK_MODEL"]
RESPON_MODEL = config["RESPON_MODEL"]
MERGE = config["MERGE"]
API_KEY = config["API_KEY"]

WORD_LIMIT = config["WORD_LIMIT"]
LEVEL_INPUT = config["LEVEL_INPUT"]
LEVEL_VALUES = config["LEVEL_VALUES"]

SEARCH_ENGINE = faiss.IndexFlatIP

In [ ]:
path = dcmt_path
Contents = LEVEL_VALUES[-1] if LEVEL_VALUES else None
print(Contents)

In [ ]:
# EXTRACT DATA
exceptions_path = "../tests/ex.exceptions.json"
markers_path = "../tests/ex.markers.json"
status_path = "../tests/ex.status.json"
extract_path = data_foler + "_dataExtract.json"
merged_path = data_foler + "_dataMerged.json"

In [ ]:
extracted_data = B1_ExtractData.extractData(dcmt_path, exceptions_path, markers_path, status_path)
with open(extract_path, "w", encoding="utf-8") as json_file:
    json.dump(extracted_data, json_file, indent=1, ensure_ascii=False)

In [ ]:
merged_data = B2_MergeData.mergeLinesToParagraphs(extracted_data)
with open(merged_path, 'w', encoding='utf-8') as f:
    json.dump(merged_data, f, indent=1, ensure_ascii=False)

In [32]:
import json
import re

try:
    data_foler
except NameError:
    data_foler = "HNMU_Regulations"

merged_path = data_foler + "_dataMerged.json"
marklist_path = data_foler + "_marklist.jsonl"

bullet_pattern = re.compile(r"^\s*[-•●♦▪‣–—]+\s*$")

# ===== 1. Đọc dữ liệu gốc =====
with open(merged_path, "r", encoding="utf-8") as f:
    data = json.load(f)

paragraphs = data.get("paragraphs", [])
common_markers = set(data.get("general", {}).get("commonMarkers", []))

# ===== 2. Lọc bullet + chỉ giữ MarkerType thuộc common =====
raw_markers = []
for p in paragraphs:
    marker_text = p.get("MarkerText")
    marker_type = p.get("MarkerType")

    # Bỏ bullet
    if bullet_pattern.match(marker_text or "") or bullet_pattern.match(marker_type or ""):
        continue

    # Chỉ giữ nếu marker_type nằm trong commonMarkers (giữ nguyên khoảng trắng)
    if marker_type in common_markers:
        raw_markers.append(marker_type)
    elif marker_type is None and None in common_markers:
        raw_markers.append(None)

# ===== 3. Loại bỏ MarkerType trùng kề nhau =====
cleaned = []
prev = object()  # sentinel
for mtype in raw_markers:
    if mtype != prev:
        cleaned.append(mtype)
        prev = mtype

# ===== 4. Xuất trực tiếp ra JSONL =====
with open(marklist_path, "w", encoding="utf-8") as f:
    for marker_type in cleaned:
        record = {"MarkerType": marker_type}
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print(f"✅ Đã xử lý xong, kết quả lưu tại {marklist_path}")


✅ Đã xử lý xong, kết quả lưu tại HNMU_Regulations_marklist.jsonl


In [33]:
import json
from collections import Counter

try:
    data_foler
except NameError:
    data_foler = "HNMU_Regulations"

marklist_path = data_foler + "_marklist.jsonl"
struct_path = data_foler + "_structures.jsonl"

with open(marklist_path, "r", encoding="utf-8") as f:
    cleaned = [json.loads(line) for line in f]

marker_types = [item.get("MarkerType", None) for item in cleaned]

results = {}

# --- Bậc 1 ---
unique_markers = list(dict.fromkeys(marker_types))
counter1 = Counter(marker_types)
results[1] = [{"Structure": [str(m)], "Count": counter1[m]} for m in unique_markers]

print("=== Bậc 1 ===")
for r in results[1]:
    print(r["Structure"], "| Xuất hiện:", r["Count"])

# --- Bậc >= 2 ---
max_depth = len(unique_markers)
prev_structures = set([tuple([m]) for m in unique_markers])

for i in range(2, max_depth + 1):
    counter = Counter()
    for j in range(len(marker_types) - i + 1):
        seq = tuple(marker_types[j:j+i])
        prefix = seq[:-1]

        if prefix not in prev_structures:
            continue

        if len(seq) != len(set(seq)):
            continue

        counter[seq] += 1

    if not counter:
        break

    min_count = min(counter.values())
    max_count = max(counter.values())

    # Giữ lại: tất cả trừ cấu trúc min (nếu nó khác max)
    filtered = {s: f for s, f in counter.items() if not (f == min_count and f != max_count)}

    # Sắp xếp theo Count giảm dần
    sorted_structs = sorted(filtered.items(), key=lambda x: x[1], reverse=True)

    results[i] = [{"Structure": [str(x) for x in s], "Count": f} for s, f in sorted_structs]

    print(f"\n=== Bậc {i} ===")
    for s, f in sorted_structs:
        print(" → ".join(str(x) for x in s), "| Xuất hiện:", f)

    # Cập nhật prev_structures
    prev_structures = set([tuple(r["Structure"]) for r in results[i]])


with open(struct_path, "w", encoding="utf-8") as f:
    for depth, structs in results.items():
        for s in structs:
            line = {
                "Depth": depth,
                "Structure": s["Structure"],
                "Count": s["Count"]
            }
            f.write(json.dumps(line, ensure_ascii=False) + "\n")

print("\n✅ Đã lưu kết quả vào HNMU_Regulations_structures.jsonl")


=== Bậc 1 ===
['Điều 123. '] | Xuất hiện: 34
['Chương XVI'] | Xuất hiện: 5
['123. '] | Xuất hiện: 67
['abc) '] | Xuất hiện: 45
['ABC: '] | Xuất hiện: 3

=== Bậc 2 ===
123.  → abc)  | Xuất hiện: 43
abc)  → 123.  | Xuất hiện: 34
Điều 123.  → 123.  | Xuất hiện: 32
123.  → Điều 123.  | Xuất hiện: 21
abc)  → Điều 123.  | Xuất hiện: 7
Chương XVI → Điều 123.  | Xuất hiện: 5
123.  → Chương XVI | Xuất hiện: 3
abc)  → ABC:  | Xuất hiện: 3
ABC:  → abc)  | Xuất hiện: 2

=== Bậc 3 ===
Điều 123.  → 123.  → abc)  | Xuất hiện: 19
abc)  → 123.  → Điều 123.  | Xuất hiện: 10
123.  → abc)  → Điều 123.  | Xuất hiện: 7
abc)  → Điều 123.  → 123.  | Xuất hiện: 6
Chương XVI → Điều 123.  → 123.  | Xuất hiện: 5
123.  → Chương XVI → Điều 123.  | Xuất hiện: 3
Điều 123.  → 123.  → Chương XVI | Xuất hiện: 2

=== Bậc 4 ===
Chương XVI → Điều 123.  → 123.  → abc)  | Xuất hiện: 3

✅ Đã lưu kết quả vào HNMU_Regulations_structures.jsonl
